In [2]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.fasttext import FastText
from string import punctuation
from nltk.corpus import stopwords
import re
punct = punctuation+'«»—…“”*№–'
stops = stopwords.words('english')
from collections import Counter,defaultdict
from gensim.models.fasttext import load_facebook_model
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/natalia-s/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def normalize(text):
    words = [word.strip(punct) for word in word_tokenize(text.lower())]
    text = ' '.join(words)
    text = re.sub(r'[\d+<>\&\$\^@#¯\(\)\{\}\[\]"©⊗→=\+μ∞×ℬℳℋπ]', '', text)
    text = re.sub(r'[\\\|\/;]', ' ', text) 
    return text

In [4]:
data = pd.read_csv("abstracts_keywords/node_attribute_abstract.csv")

In [9]:
nodes = list(data.node)

In [11]:
import json
id_map = {nodes[i]: i for i in range(len(nodes))}
with open('abstract-id_map.json', 'w') as f:
    f.write(json.dumps(id_map))

In [ ]:
data['clean'] = data['abstract'].apply(lambda x: normalize(x))

In [ ]:
ft = load_facebook_model('cc.en.300.bin')

In [ ]:
#функция для поиска векторов в предобученных векторах Fasttext
def getWordVecs(text,dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    for i,word in enumerate(words):
        try:
            v = ft[word]
            vectors[i] = v*(words[word]/total) 
        except (KeyError, ValueError):
            continue
    
    if vectors.any(): #усредняем векторы
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))

            
    return vector 


In [ ]:
def vectorize(texts, dim):
    vects = np.zeros((len(texts), dim))
    for i, text in enumerate(texts.values):
        vects[i] = getWordVecs(text,dim)
        
    return  vects   

In [ ]:
scipy.sparse.save_npz('vects.npz', sparse_matrix)